In [ ]:
from qiskit import QuantumCircuit, Aer
from qiskit.circuit import Parameter
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
import torch
import torch.nn as nn
import numpy as np

# Define a simple variational quantum circuit with one qubit
theta = Parameter("θ")
qc = QuantumCircuit(1)
qc.h(0)  # Superposition
qc.rx(theta, 0)  # Trainable rotation

# Quantum Neural Network using Qiskit's EstimatorQNN
quantum_nn = EstimatorQNN(qc, input_params=[], weight_params=[theta])

# Connect to PyTorch
qnn_model = TorchConnector(quantum_nn)

# Define a simple classical neural network with QNN layer
class HybridQNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.qnn_layer = qnn_model
        self.fc = nn.Linear(1, 1)  # Fully connected layer

    def forward(self, x):
        x = self.qnn_layer(x)
        return self.fc(x)

# Create model and optimizer
model = HybridQNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
loss_function = nn.MSELoss()

# Training Data (Simple XOR-like pattern)
X = torch.tensor([[0.0], [1.0]], dtype=torch.float32)
Y = torch.tensor([[0.0], [1.0]], dtype=torch.float32)

# Training loop
for epoch in range(100):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_function(output, Y)
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item()}")

# Test the trained model
with torch.no_grad():
    test_data = torch.tensor([[0.5]], dtype=torch.float32)
    prediction = model(test_data)
    print(f"Prediction for input 0.5: {prediction.item()}")


In [ ]:
from qiskit.visualization import circuit_drawer

# Display the quantum circuit
print(qc.draw())
circuit_drawer(qc, output="mpl")


In [ ]:
from qiskit.circuit.library import RealAmplitudes

# Define a multi-qubit quantum circuit (2 qubits, 2 layers)
num_qubits = 2
qc_multi = RealAmplitudes(num_qubits, entanglement='full', reps=2)

# Visualize the circuit
qc_multi.decompose().draw("mpl")


In [ ]:
from qiskit_machine_learning.neural_networks import SamplerQNN

# Define a QNN for classification using the multi-qubit circuit
quantum_classifier = SamplerQNN(qc_multi)

# Connect to PyTorch
qnn_classifier = TorchConnector(quantum_classifier)

# Define a simple PyTorch model with a QNN-based layer
class QuantumClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.qnn_layer = qnn_classifier
        self.fc = nn.Linear(4, 1)  # 4 output probabilities -> 1 class

    def forward(self, x):
        x = self.qnn_layer(x)
        return torch.sigmoid(self.fc(x))  # Binary classification

# Initialize model
classifier = QuantumClassifier()


In [ ]:
# XOR classification dataset
X_train = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
Y_train = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)  # XOR labels

# Training loop
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.1)
loss_function = nn.BCELoss()  # Binary Cross Entropy

for epoch in range(200):
    optimizer.zero_grad()
    output = classifier(X_train)
    loss = loss_function(output, Y_train)
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item()}")

# Testing
test_input = torch.tensor([[1, 0]], dtype=torch.float32)
prediction = classifier(test_input).item()
print(f"Prediction for input [1, 0]: {prediction:.4f}")
